<a href="https://colab.research.google.com/github/NavaneethRaj17/AI_Research_Agent/blob/Init/AI_Research_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install streamlit  ## to build web app
#!pip install openai  ## to use GPT-4o-mini
#!pip install mem0ai  ## to access Arxiv(Internet of Agent)
#!pip install multion  ## Personalized memory layer

In [2]:
import streamlit as st
import os
from mem0 import Memory
from multion.client import MultiOn
from openai import OpenAI

In [3]:
# Setting up Streamlit app
st.title("AI Research Agent with Memory")
api_keys = {k: st.text_input(f"{k.capitalize()} API Key", type="password") for k in ['openai','multion']} #description for the app

2024-09-25 08:31:20.442 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.715 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-09-25 08:31:20.717 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.721 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.723 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.725 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.726 Session state does not function when running a script without `streamlit run`
2024-09-25 08:31:20.728 Thread 'MainThread': missing ScriptRunContext! 

In [4]:
# Initialize services if API keys are provided
if all(api_keys.values()):      #Configures Mem0 with Qdrant as the vector store
  os.environ['OPENAI_API_KEY'] = api_keys['openai']
  config = {
      "vector_store":{
          "provider":"qdrant",
          "config":{
              "model":"gpt-4o-mini",
              "host":"localhost",
              "port":6333,
              "collection_name":"arxiv"

          }
      }
  }
  memory= Memory.from_config(config)  #initialize MultiOn and OpenAI clients
  multion= Multion(api_key=api_keys['multion'])
  openai_client = OpenAI(api_key=api_keys['openai'])

In [5]:
# Create user input and search query fields
user_id= st.sidebar.text_input("Enter your username") #adds a sidebar for user identification
search_query= st.text_input("Research paper search query") #input field for research paper search query

2024-09-25 08:31:20.786 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.788 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.792 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.795 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.797 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.799 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.803 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.805 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [6]:
# Function to process search results with GPT-4o_mini
def process_with_gpt4(result):
  prompt = f"""
  Based on the following arXiv search result, provide a proper structured output in markdown that is readable by the users.
  Each paper should have a title,authors,abstract and link.
  Search Result: {result}
  output Format: Table with the following columns: [{{"title":"Paper Title","authors":"Author Names","abstract":"Brief abstract","link":"arXiv link"}}, ...]
  """
  response= openai_client.chat.completions.create(model="gpt-4o-mini",messages=[{"role":"user","content":prompt}],temperature=0.2)
  return response.choices[0].message.content

In [7]:
# Implement the paper search functionality
if st.button('Search for Papers'):
  with st.spinner("Searching and Processing..."):
    relevant_memories = memory.search(search_query,user_id=user_id,limit=3)
    prompt = f"Search for arXiv papers: {search_query}\n User background: {' '.join(mem['text'] for mem in relevant_memories)}"
    result= process_with_gpt4(multion.browse(cmd=prompt,url="https://arxiv.org/"))
    st.markdown(result)


2024-09-25 08:31:20.841 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.845 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.847 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.848 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
# Adding a memory viewing feature
if st.sidebar.button("View Memory"):
    st.sidebar.write("\n".join([f"- {mem['text']}" for mem in memory.get_all(user_id= user_id)]))

2024-09-25 08:31:20.865 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.868 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.870 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-25 08:31:20.872 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
